In [3]:
%pip install google-cloud-aiplatform

  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.28.3
    Uninstalling protobuf-5.28.3:
      Successfully uninstalled protobuf-5.28.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.38.0 requires tenacity<9,>=8.1.0, but you have tenacity 9.0.0 which is incompatible.
grpcio-health-checking 1.68.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 4.25.6 which is incompatible.
opentelemetry-exporter-otlp-proto-http 1.26.0 requires opentelemetry-exporter-otlp-proto-common==1.26.0, but you have opentelemetry-exporter-otlp-proto-common 1.25.0 which is incompatible.
opentelemetry-exporter-otlp-proto-http 1.26.0 requires opentelemetry-proto==1.26.0, but you have opentelemetry-proto 1.25.0 which is incompatible.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.16.2 which is incompatible.
grpcio-to

In [1]:
from transformers import pipeline
import torch
from google.cloud import aiplatform
def summarize(text):
    # Check GPU availability and setup
    device = 0 if torch.cuda.is_available() else -1
    print(f"Using device: {'cuda' if device == 0 else 'cpu'}")
    
    # Initialize the pipeline with GPU support
    summarizer = pipeline(
        "summarization",
        model="facebook/bart-large-cnn",
        device=device,
        torch_dtype=torch.float16
    )

    # Chunk the text
    max_chunk_size = 1024
    chunks = [text[i:i+max_chunk_size] for i in range(0, len(text), max_chunk_size)]

    summaries = []
    for chunk in chunks:
        summary = summarizer(
                    chunk,
                    max_length=130,
                    min_length=30,
                    do_sample=False
                )
        summaries.append(summary[0]['summary_text'])
    return " ".join(summaries)

In [8]:
!gcloud auth login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=7tXKV3UqXdQGh9wn2yDYbevfZAA1JH&access_type=offline&code_challenge=hQc4xHukiqUtr56lVPQDk5IcOHep9olrQ407CP1FeYs&code_challenge_method=S256


You are now logged in as [galluptj@gmail.com].
Your current project is [solid-idiom-448521-c7].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


Updates are available for some Google Cloud CLI components.  To install them,
please run:
  $ gcloud components update



In [12]:
import os
os.environ['GRPC_DNS_RESOLVER'] = 'native'

In [13]:
aiplatform.init(project='solid-idiom-448521-c7', location='us-west1', staging_bucket="summarization-bucket1")
# Create the custom job
custom_job = aiplatform.CustomTrainingJob(
    display_name='summarization_gpu',
    script_path='summarize_script.py',
    container_uri="us-docker.pkg.dev/vertex-ai/training/pytorch-gpu.1-13:latest",
    requirements=['transformers', 'torch', 'google-cloud-storage'],
)

# Create the custom job
custom_job = aiplatform.CustomTrainingJob(
    display_name='summarization_gpu',
    script_path='summarize_script.py',
    container_uri="us-docker.pkg.dev/vertex-ai/training/pytorch-gpu.1-13:latest",
    requirements=['transformers', 'torch', 'google-cloud-storage'],
)

# Launch the training job
custom_job.run(
    machine_type="n1-standard-8",
    accelerator_type="NVIDIA_TESLA_T4",
    accelerator_count=1,
    replica_count=1,
    base_output_dir='gs://summarization-bucket1/output'
)

Training script copied to:
gs://summarization-bucket1/aiplatform-2025-02-01-16:25:01.709-aiplatform_custom_trainer_script-0.1.tar.gz.
Training Output directory:
gs://summarization-bucket1/output 
View Training:
https://console.cloud.google.com/ai/platform/locations/us-west1/training/1895711428157898752?project=899524345502
CustomTrainingJob projects/899524345502/locations/us-west1/trainingPipelines/1895711428157898752 current state:
3
View backing custom job:
https://console.cloud.google.com/ai/platform/locations/us-west1/training/2507497290038509568?project=899524345502
CustomTrainingJob projects/899524345502/locations/us-west1/trainingPipelines/1895711428157898752 current state:
3
CustomTrainingJob projects/899524345502/locations/us-west1/trainingPipelines/1895711428157898752 current state:
3
CustomTrainingJob projects/899524345502/locations/us-west1/trainingPipelines/1895711428157898752 current state:
3
CustomTrainingJob projects/899524345502/locations/us-west1/trainingPipelines/189

RetryError: Timeout of 120.0s exceeded, last exception: 503 failed to connect to all addresses; last error: UNKNOWN: ipv6:%5B2607:f8b0:4005:813::200a%5D:443: No route to host